In [1]:
import numpy as np
from scipy.stats import rice
from scipy.special import i0
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import matplotlib


def ivim_signal(b, S0, D, f, D_star):
    """
    Calculates the IVIM signal under the bi-exponential model.

    Args:
        b (array-like): b-values
        S0 (float): Initial signal amplitude 
        D (float): True diffusion coefficient
        f (float): Perfusion fraction
        D_star (float): Pseudodiffusion coefficient

    Returns:
        array-like: IVIM signal values
    """
    return S0 * (f * np.exp(-b * D_star) + (1 - f) * np.exp(-b * D)) 

def jacobian(b, S0, D, f, D_star):
    """
    Calculates the Jacobian matrix of the IVIM signal model.

    Args:
        b (array-like): b-values
        S0 (float): Initial signal amplitude 
        D (float): True diffusion coefficient
        f (float): Perfusion fraction
        D_star (float): Pseudodiffusion coefficient

    Returns:
        numpy.ndarray: Jacobian matrix
    """
    dSdS0 = f * np.exp(-b * D_star) + (1 - f) * np.exp(-b * D)
    dSdD = -S0 * (1 - f) * b * np.exp(-b * D)
    dSdF = S0 * (np.exp(-b * D_star) - np.exp(-b * D))
    dSdDstar = -S0 * f * b * np.exp(-b * D_star)
    return np.stack([dSdS0, dSdD, dSdF, dSdDstar], axis=1)


def crlb(b, S0, D, f, D_star, sigma):
    """
    Calculates the Cramer-Rao Lower Bound (CRLB) for IVIM parameters.
    With the assumption of Gaussian noise (so this approximation 
    does not account for noise floor effects)
    Without accounting for TE-lengthening required for higher b values

    Args:
        b (array-like): b-values
        S0 (float): Initial signal amplitude 
        D (float): True diffusion coefficient
        f (float): Perfusion fraction
        D_star (float): Pseudodiffusion coefficient
        sigma (float): Standard deviation of Rician noise

    Returns:
        numpy.ndarray: Covariance matrix (the diagonal elements are CRLBs)
    """
    J = jacobian(b, S0, D, f, D_star)
    fisher_info = (J.T @ J) / sigma**2 
    return np.linalg.inv(fisher_info)  

# Example usage:
b_values = np.array([0, 10, 100, 200, 500, 800])
true_S0 = 1.0  # mm^2/s
true_D = 0.001  # mm^2/s
true_f = 0.15
true_D_star = 0.02  # mm^2/s
sigma = 0.05  # Standard deviation of noise

crlb_matrix = crlb(b_values, true_S0, true_D, true_f, true_D_star, sigma)

std_D = np.sqrt(crlb_matrix[1,1]);
std_f = np.sqrt(crlb_matrix[2,2]);
std_D_star = np.sqrt(crlb_matrix[3,3]);

#print("Sqrt CRLB for D:", std_D)
#print("Sqrt CRLB for f:", std_f)
#print("Sqrt CRLB for D*:", std_D_star) 


error_metric = std_D/true_D + std_f +  std_D_star/true_D_star # Not normalizing by true_f, which can be zero
print("Error metric", error_metric) 


Error metric 2.514208947502162


In [3]:
# Cost function that penalizes standard deviations predicted by the CRLB
def objective_function_range(b, S0s, Ds, fs, D_stars, sigma):       
    b = np.clip(b,0,1500)
    error_metric = 0.0
    error_metrics = []
    for S0 in S0s:
        for D in Ds:
            for f in fs:
                for D_star in D_stars:
                    crlb_matrix = crlb(b, S0, D, f, D_star, sigma)
                    std_D = np.sqrt(crlb_matrix[1,1]);
                    std_f = np.sqrt(crlb_matrix[2,2]);
                    std_D_star = np.sqrt(crlb_matrix[3,3]);
                    cur_error =  std_D/D + std_f +  std_D_star/D_star # Not normalizing by true_f, which can be zero
                    error_metrics = np.append(error_metrics,(cur_error))
    # Maximum error across all true parameter settings (could also take the average, or some other combination)
    error_metric = np.max(error_metrics)
    return error_metric


# Measures of uncertainty for each of the IVIM parameters 
def uncertainty_measures(b, S0s, Ds, fs, D_stars, sigma):       
    b = np.clip(b,0,1500)
    error_metric = 0.0
    error_metrics = []
    error_metrics_D = []
    error_metrics_f = []
    error_metrics_D_star = []
    for S0 in S0s:
        for D in Ds:
            for f in fs:
                for D_star in D_stars:
                    crlb_matrix = crlb(b, S0, D, f, D_star, sigma)
                    std_D = np.sqrt(crlb_matrix[1,1]);
                    std_f = np.sqrt(crlb_matrix[2,2]);
                    std_D_star = np.sqrt(crlb_matrix[3,3]);
                    cur_error =  std_D/D + std_f +  std_D_star/D_star # Not normalizing by true_f, which can be zero
                    error_metrics_D = np.append(error_metrics_D,(std_D/D))
                    error_metrics_D_star = np.append(error_metrics_D_star,(std_D_star/D_star))
                    error_metrics_f = np.append(error_metrics_f,(std_f))
                    error_metrics = np.append(error_metrics,(cur_error))
    ## Maximum error across all true parameter settings (could also take the average, or some other combination)
    ##error_metric = np.max(error_metrics)
    # Average error across all true parameter settings 
    error_metric = np.mean(error_metrics)
    return error_metric, error_metrics_D, error_metrics_f, error_metrics_D_star



In [5]:
# True parameters from Eric's excel sheet (July 2024)
trueParams= np.dtype([('name', (np.str_, 30)), ('D', np.float64),('f', np.float64), ('Dstar', np.float64), 
                      ('SD_D', np.float64),('SD_f', np.float64), ('SD_Dstar', np.float64)]) 
myTrueParams = np.array([('Kidney Cortex 1.5T', 1.966/1000, 19.9/100, 50.8/1000, 0.075/1000, 3.2/100, 13.45/1000), 
                         ('Kidney Cortex 3.0T', 1.919/1000, 20.1/100, 24.964/1000, 0.229/1000, 8.4/100,20.3/1000 ), 
                         ('Kidney Medulla 1.5T', 1.884/1000, 17.5/100, 57.35/1000, 0.076/1000, 5.5/100, 25.5/1000),
                         ('Kidney Medulla 3.0T', 1.796/1000, 18/100, 29.02/1000, 0.228/1000, 7.8/100, 19.27/1000),
                         ('Kidney Average', 1.891/1000, 18.875/100, 40.534/1000, 0.152/1000, 6.225/100, 19.63/1000),
                         ('Liver', 1.09/1000, 23.05/100, 70.02/1000,0.17/1000, 8.48/100, 31.01/1000),
                         ('Muscle', 1.47/1000, 10.34/100, 30.88/1000, 0.28/1000, 5.92/100, 39.18/1000 ),
                         ('Breast Malignant', 0.9676/1000, 11.305/100, 37.76/1000, 0.32/1000, 4.506/100, 19.12/1000),
                         ('Breast Benign', 1.4305/1000, 7.004/100, 52.33/1000, 0.37/1000, 4.22/100, 28.50/1000),
                         ('Pancreas Malignant', 1.396/1000, 12.394/100, 22.16/1000,0.48/1000, 4.97/100, 12.83/1000),
                         ('Pancreas Benign', 1.409/1000, 20.033/100, 25.39/1000,0.72/1000, 7.99/100, 13.69/1000)
                        ], 
       dtype=trueParams) 


In [7]:
# For each example of tissue parameters, optimize the b values
num_b = 6
allb = np.zeros([myTrueParams.size,num_b])
for k in range(myTrueParams.size):

    # Collect current tissue parameters
    print("Current tissue parameters:", myTrueParams[k])
    true_S0s = np.array([1.0])
    true_Ds = np.array([myTrueParams[k]['D']])
    true_fs = np.array([myTrueParams[k]['f']])
    true_D_stars = np.array([myTrueParams[k]['Dstar']])
    sigma = 0.04  # Standard deviation of noise (assuming SNR = true_S0s / sigma for the b=0 signal)

    # Define fixed parameters for optimization
    fixed_params = {'S0s': true_S0s, 'Ds': true_Ds, 'fs': true_fs, 'D_stars':true_D_stars, 'sigma':sigma}

    # Choose an optimization method and provide an initial guess for x
    method = 'Nelder-Mead'  # Other options: 'differential_evolution', etc.
    b0 = np.logspace(0, 2.5, num_b, base=10.0)

    #print("Initial b values:", np.round(b0))

    # Minimize the objective function (using args to pass fixed parameters)
    result = minimize(objective_function_range, b0, method=method, args=(fixed_params['S0s'], fixed_params['Ds'], fixed_params['fs'], fixed_params['D_stars'], fixed_params['sigma']),options={'maxiter': 50000, 'maxfev': 200000})
    b_opt = np.clip(result.x,0,2500)
    b0 = b_opt
    result = minimize(objective_function_range, b0, method=method, args=(fixed_params['S0s'], fixed_params['Ds'], fixed_params['fs'], fixed_params['D_stars'], fixed_params['sigma']),options={'maxiter': 50000, 'maxfev': 200000})
    b_opt = np.clip(result.x,0,2500)
    b_opt = np.sort(b_opt)

    # Check if the optimization was successful
    if not result.success:
        print("Optimization failed!")

    print("Optimized b values:", np.round(b_opt))
    allb[k,:] = b_opt

    # Calculate some measure of uncertainty for each of the parameters in each of the organs
    # Note that these measures of uncertainty scale with the noise assumed in the signal, so the absolute values need to be interpreted with care
    error_metric, error_metrics_D, error_metrics_f, error_metrics_D_star = uncertainty_measures(b_opt, true_S0s, true_Ds, true_fs, true_D_stars, sigma)
    print("Uncertainty for D (stdD/D):",  "{:.2f}".format(np.mean(error_metrics_D)))
    print("Uncertainty for f (stdf):",  "{:.2f}".format(np.mean(error_metrics_f)))
    print("Uncertainty for Dstar (stdDstar/Dstar):",  "{:.2f}".format(np.mean(error_metrics_D_star)))
    


Current tissue parameters: ('Kidney Cortex 1.5T', 0.001966, 0.199, 0.0508, 7.5e-05, 0.032, 0.01345)
Optimized b values: [  0.  16.  16.  16.  97. 742.]
Uncertainty for D (stdD/D): 0.18
Uncertainty for f (stdf): 0.07
Uncertainty for Dstar (stdDstar/Dstar): 0.63
Current tissue parameters: ('Kidney Cortex 3.0T', 0.001919, 0.201, 0.024964, 0.000229, 0.084, 0.0203)
Optimized b values: [  0.   0.  28.  28. 164. 858.]
Uncertainty for D (stdD/D): 0.21
Uncertainty for f (stdf): 0.10
Uncertainty for Dstar (stdDstar/Dstar): 0.76
Current tissue parameters: ('Kidney Medulla 1.5T', 0.001884, 0.175, 0.05735, 7.6e-05, 0.055, 0.0255)
Optimized b values: [  0.   0.  13.  13.  91. 762.]
Uncertainty for D (stdD/D): 0.17
Uncertainty for f (stdf): 0.07
Uncertainty for Dstar (stdDstar/Dstar): 0.71
Current tissue parameters: ('Kidney Medulla 3.0T', 0.001796, 0.18, 0.02902, 0.000228, 0.078, 0.01927)
Optimized b values: [  0.   0.  25.  25. 151. 884.]
Uncertainty for D (stdD/D): 0.19
Uncertainty for f (stdf): 0

In [9]:
allb = np.round(allb*10)/10
for k in range(myTrueParams.size):
    print(allb[k,0],  allb[k,1],allb[k,2],allb[k,3] )


0.0 15.8 15.8 15.8
0.0 0.0 27.5 27.5
0.0 0.0 13.4 13.4
0.0 0.0 24.5 24.5
0.0 0.0 18.2 18.2
0.0 12.3 12.3 12.3
0.0 0.0 23.9 23.9
0.0 0.0 20.8 20.8
0.0 0.0 14.9 14.9
0.0 0.0 0.0 33.9
0.0 0.0 28.4 28.4


In [19]:
# For each example of tissue parameters, optimize the b values 
# In this case, we will include a range of D, f, and D* values in the optimization
# based on the SDs provided in Eric's excel sheet
num_b = 6
allb = np.zeros([myTrueParams.size,num_b])
for k in range(myTrueParams.size):

    # Collect current tissue parameters
    print("Current tissue parameters:", myTrueParams[k])

    # Set the ranges of true tissue parameters to include, based on the mean and SD for each 
    # Basically, the ranges are (mean-SD, mean+SD) for each parameter, with some sanity checks
    curD = myTrueParams[k]['D']
    curSD_D = myTrueParams[k]['SD_D']
    curf = myTrueParams[k]['f']
    curSD_f = myTrueParams[k]['SD_f']
    curDstar = myTrueParams[k]['Dstar']
    curSD_Dstar = myTrueParams[k]['SD_Dstar']
    minD = max(0.0,curD-curSD_D)
    maxD = curD+curSD_D
    minf = max(0.0,curf-curSD_f)
    maxf = min(1.0,curf+curSD_f)
    minDstar = max(10/1000,curDstar-curSD_Dstar) # Let us avoid very small Dstar values, which complicate IVIM fitting
    maxDstar = curDstar+curSD_Dstar

    # Define some arrays with the true tissue parameters for the CRLB optimization
    true_S0s = np.array([1.0])
    true_Ds = np.linspace(minD, maxD, num=5, endpoint=True)
    true_fs = np.linspace(minf, maxf, num=3, endpoint=True)
    true_D_stars = np.linspace(minDstar, maxDstar, num=11, endpoint=True)
    sigma = 0.04  # Standard deviation of noise (assuming SNR = true_S0s / sigma for the b=0 signal)

    # Define fixed parameters for optimization
    fixed_params = {'S0s': true_S0s, 'Ds': true_Ds, 'fs': true_fs, 'D_stars':true_D_stars, 'sigma':sigma}

    # Choose an optimization method and provide an initial guess for x
    method = 'Nelder-Mead'  # Other options: 'differential_evolution', etc.
    b0 = np.logspace(0, 2.5, num_b, base=10.0)

    #print("Initial b values:", np.round(b0))

    # Minimize the objective function (using args to pass fixed parameters)
    result = minimize(objective_function_range, b0, method=method, args=(fixed_params['S0s'], fixed_params['Ds'], fixed_params['fs'], fixed_params['D_stars'], fixed_params['sigma']),options={'maxiter': 50000, 'maxfev': 200000})
    b_opt = np.clip(result.x,0,2500)
    b0 = b_opt
    result = minimize(objective_function_range, b0, method=method, args=(fixed_params['S0s'], fixed_params['Ds'], fixed_params['fs'], fixed_params['D_stars'], fixed_params['sigma']),options={'maxiter': 50000, 'maxfev': 200000})
    result = minimize(objective_function_range, result.x, method=method, args=(fixed_params['S0s'], fixed_params['Ds'], fixed_params['fs'], fixed_params['D_stars'], fixed_params['sigma']),options={'maxiter': 50000, 'maxfev': 200000})
    result = minimize(objective_function_range, result.x, method=method, args=(fixed_params['S0s'], fixed_params['Ds'], fixed_params['fs'], fixed_params['D_stars'], fixed_params['sigma']),options={'maxiter': 50000, 'maxfev': 200000})
    b_opt = np.clip(result.x,0,2500)
    b_opt = np.sort(b_opt)

    # Check if the optimization was successful
    if not result.success:
        print("Optimization failed!")

    print("Optimized b values:", np.round(b_opt))
    allb[k,:] = b_opt

    # Calculate some measure of uncertainty for each of the parameters in each of the organs
    # Note that these measures of uncertainty scale with the noise assumed in the signal, so the absolute values need to be interpreted with care
    error_metric, error_metrics_D, error_metrics_f, error_metrics_D_star = uncertainty_measures(b_opt, true_S0s, true_Ds, true_fs, true_D_stars, sigma)
    print("Uncertainty for D (stdD/D):",  "{:.2f}".format(np.mean(error_metrics_D)))
    print("Uncertainty for f (stdf):",  "{:.2f}".format(np.mean(error_metrics_f)))
    print("Uncertainty for Dstar (stdDstar/Dstar):",  "{:.2f}".format(np.mean(error_metrics_D_star)))
    


Current tissue parameters: ('Kidney Cortex 1.5T', 0.001966, 0.199, 0.0508, 7.5e-05, 0.032, 0.01345)
Optimized b values: [  0.  19.  19.  19. 114. 759.]
Uncertainty for D (stdD/D): 0.18
Uncertainty for f (stdf): 0.08
Uncertainty for Dstar (stdDstar/Dstar): 0.67
Current tissue parameters: ('Kidney Cortex 3.0T', 0.001919, 0.201, 0.024964, 0.000229, 0.084, 0.0203)
Optimized b values: [   0.    0.   40.   65.  292. 1040.]
Uncertainty for D (stdD/D): 0.25
Uncertainty for f (stdf): 0.15
Uncertainty for Dstar (stdDstar/Dstar): 1.42
Current tissue parameters: ('Kidney Medulla 1.5T', 0.001884, 0.175, 0.05735, 7.6e-05, 0.055, 0.0255)
Optimized b values: [  0.   0.  15.  17. 122. 814.]
Uncertainty for D (stdD/D): 0.18
Uncertainty for f (stdf): 0.07
Uncertainty for Dstar (stdDstar/Dstar): 0.85
Current tissue parameters: ('Kidney Medulla 3.0T', 0.001796, 0.18, 0.02902, 0.000228, 0.078, 0.01927)
Optimized b values: [   0.    0.   39.   67.  301. 1085.]
Uncertainty for D (stdD/D): 0.23
Uncertainty for